In [1]:
from LogisticRegression import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



In [2]:
df = pd.read_csv('clean_data.csv')

In [3]:
selected_features = [
    'is_canceled',
    'hotel',
    'stays_in_weekend_nights',
    'adults',
    'children',
    'market_segment',
    'total_of_special_requests',
    'is_repeated_guest',
    'previous_cancellations',
    'previous_bookings_not_canceled',
    'deposit_type',
    'total_orang'
]

selected_df = df[selected_features]

label_encoder = LabelEncoder()

categorical_columns = ['hotel', 'market_segment', 'deposit_type']
for col in categorical_columns:
    selected_df[col] = label_encoder.fit_transform(selected_df[col])

X = selected_df.drop(columns=['is_canceled']).values
y = selected_df['is_canceled'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

C:\Users\fadhl\AppData\Local\Temp\ipykernel_30696\1334936061.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df[col] = label_encoder.fit_transform(selected_df[col])
C:\Users\fadhl\AppData\Local\Temp\ipykernel_30696\1334936061.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df[col] = label_encoder.fit_transform(selected_df[col])
C:\Users\fadhl\AppData\Local\Temp\ipykernel_30696\1334936061.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [4]:
model = LogisticRegression(learning_rate=1e-5 ,ephocs=10000)

model.fit(X_train, y_train)

In [5]:
y_predict = model.predict(X_test)

In [6]:
from sklearn.metrics import classification_report


report = classification_report(y_test, y_predict, target_names=['Class 0', 'Class 1'])

print(report)

              precision    recall  f1-score   support

     Class 0       0.71      1.00      0.83      9905
     Class 1       0.00      0.00      0.00      4078

    accuracy                           0.71     13983
   macro avg       0.35      0.50      0.41     13983
weighted avg       0.50      0.71      0.59     13983



c:\Program Files\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Program Files\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Program Files\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
for i in range(len(y_predict)):
    if y_predict[i]==1:
        print(i)

In [9]:
model.weights

array([0.00988897, 0.02149613])